In [60]:
import parsimonious
from FileParser import FileParser
from ModelParser import ModelParser
from ComponentParser import ComponentParser
from ModelObjects import Element, Stock, ValueElement, Flow, Variable, ValueType, ValueTypeConstant, Source, Sink
import re

In [24]:
# Read in the model
mdl_file = 'ASD_V2.mdl'

with open(mdl_file, 'r') as in_file:
        text = in_file.read()   

file_str = text.replace('\n', '')

In [29]:
def get_file_sections(file_str):
    file_structure_grammar = r"""
    file = encoding? (macro / main)+
    macro = ":MACRO:" _ name _ "(" _ (name _ ","? _)+ _ ":"? _ (name _ ","? _)* _ ")" ~r".+?(?=:END OF MACRO:)" ":END OF MACRO:"
    main = !":MACRO:" ~r".+(?!:MACRO:)"

    name = basic_id / escape_group
    basic_id = ~r"[a-zA-Z][a-zA-Z0-9_\s]*"

    # between quotes, either escaped quote or character that is not a quote
    escape_group = "\"" ( "\\\"" / ~r"[^\"]" )* "\""
    encoding = ~r"\{[^\}]*\}"

    _ = ~r"[\s\\]*"  # whitespace character
    """  # the leading 'r' for 'raw' in this string is important for handling backslashes properly

    parser = parsimonious.Grammar(file_structure_grammar)
    tree = parser.parse(file_str)
    
    return FileParser(tree).entries

In [37]:
def get_model_elements(model_str):
    model_structure_grammar = r"""
    model = (entry / section)+ sketch?
    entry = element "~" element "~" element ("~" element)? "|"
    section = element "~" element "|"
    sketch = ~r".*"  #anything

    # Either an escape group, or a character that is not tilde or pipe
    element = (escape_group / ~r"[^~|]")*

    # between quotes, either escaped quote or character that is not a quote
    escape_group = "\"" ( "\\\"" / ~r"[^\"]" )* "\""
    """
    parser = parsimonious.Grammar(model_structure_grammar)
    tree = parser.parse(model_str)
    #print(tree)

    return ModelParser(tree).entries

In [38]:
def get_equation_components(equation_str):
    component_structure_grammar = r"""
    entry = component / subscript_definition / lookup_definition
    component = name _ subscriptlist? _ "=" _ expression
    subscript_definition = name _ ":" _ subscript _ ("," _ subscript)*
    lookup_definition = name _ &"(" _ expression  # uses lookahead assertion to capture whole group

    name = basic_id / escape_group
    subscriptlist = '[' _ subscript _ ("," _ subscript)* _ ']'
    expression = ~r".*"  # expression could be anything, at this point.

    subscript = basic_id / escape_group

    basic_id = ~r"[a-zA-Z][a-zA-Z0-9_\s]*"
    escape_group = "\"" ( "\\\"" / ~r"[^\"]" )* "\""
    _ = ~r"[\s\\]*"  # whitespace character
    """

    # replace any amount of whitespace  with a single space
    equation_str = equation_str.replace('\\t', ' ')
    equation_str = re.sub(r"\s+", ' ', equation_str)

    parser = parsimonious.Grammar(component_structure_grammar)
    tree = parser.parse(equation_str)
    print(tree)

    

    parse_object = ComponentParser(tree)

    return {'real_name': parse_object.real_name,
            'subs': parse_object.subscripts,
            'expr': parse_object.expression,
            'kind': parse_object.kind}


In [43]:
# Go through every file section and translate it
file_sections = get_file_sections(file_str)
print(len(file_sections))

for section in file_sections:
    model_elements = get_model_elements(section['string'])
    model_docstring = ''    
print(len(model_elements)) 


for entry in model_elements:
    if entry['kind'] == 'entry':
        entry.update(get_equation_components(entry['eqn']))

print(model_elements)

1
17
<Node called "entry" matching "percent of muslims in population= 0.5">
    <Node called "component" matching "percent of muslims in population= 0.5">
        <Node called "name" matching "percent of muslims in population">
            <RegexNode called "basic_id" matching "percent of muslims in population">
        <RegexNode called "_" matching "">
        <Node matching "">
        <RegexNode called "_" matching "">
        <Node matching "=">
        <RegexNode called "_" matching " ">
        <RegexNode called "expression" matching "0.5">
<Node called "entry" matching "percent with access to internet= 0.5">
    <Node called "component" matching "percent with access to internet= 0.5">
        <Node called "name" matching "percent with access to internet">
            <RegexNode called "basic_id" matching "percent with access to internet">
        <RegexNode called "_" matching "">
        <Node matching "">
        <RegexNode called "_" matching "">
        <Node matching "=">


In [50]:
stocks = []
potential_flows = []
for element in model_elements:
    if 'INTEG' in element['eqn']:
        stocks.append(element)
        integ_index = element['eqn'].find("INTEG (") + 7
        comma_index = element['eqn'].find(",")
        potential_flows.append(element['eqn'][integ_index:comma_index])


[{'kind': 'component', 'doc': '', 'unit': 'person', 'eqn': 'radicalized citizens= INTEG (\tpersuasion-deradicalization,\t\t0.005*total population)', 'real_name': 'radicalized citizens', 'subs': [], 'expr': 'INTEG ( persuasion-deradicalization, 0.005*total population)'}, {'kind': 'component', 'doc': '', 'unit': 'person', 'eqn': 'exposed citizens on internet= INTEG (\tderadicalization+exposure-persuasion,\t\t0)', 'real_name': 'exposed citizens on internet', 'subs': [], 'expr': 'INTEG ( deradicalization+exposure-persuasion, 0)'}, {'kind': 'component', 'doc': '', 'unit': 'person [0,?]', 'eqn': 'unexposed citizens on internet= INTEG (\t-exposure,\t\ttotal population*percent with access to internet)', 'real_name': 'unexposed citizens on internet', 'subs': [], 'expr': 'INTEG ( -exposure, total population*percent with access to internet)'}]
['\tpersuasion-deradicalization', '\tderadicalization+exposure-persuasion', '\t-exposure']


In [57]:
flows = []
variables = []

for element in model_elements:
    found_flow = False
    for potential_flow in potential_flows:
        if found_flow == False and element['real_name'] in potential_flow:
            flows.append(element)
            found_flow = True
    if found_flow == False and 'INTEG' not in element['eqn']:
        variables.append(element)

print(list(map(lambda f: f['real_name'],stocks)))
print(list(map(lambda f: f['real_name'],flows)))
print(list(map(lambda f: f['real_name'],variables)))

['radicalized citizens', 'exposed citizens on internet', 'unexposed citizens on internet']
['exposure', 'persuasion', 'deradicalization']
['percent of muslims in population', 'percent with access to internet', 'number of foreign fighters', 'persuasion rate', 'deradicalization rate', 'exposure rate', 'total population', 'FINAL TIME', 'INITIAL TIME', 'SAVEPER', 'TIME STEP']


In [63]:
elements = []
stock_mapping = {}

for stock in stocks:
    el = Stock(stock['real_name'],ValueType('0',ValueTypeConstant.constant))
    elements.append(el)
    stock_mapping[stock['real_name']] = el
for variable in variables:
    elements.append(Variable(variable['real_name'],ValueType('0',ValueTypeConstant.constant)))
for flow in flows:
    src = Source('src')
    dst = Sink('dst')
    flow_name = flow['real_name']
    for stock in stocks:
        if '-'+flow_name in stock['eqn']:
            src = stock_mapping[stock['real_name']]
        elif flow_name in stock['eqn']:
            dst = stock_mapping[stock['real_name']]
    elements.append(Flow(flow_name,ValueType('0',ValueTypeConstant.constant),src, dst))

print(elements)
    

[<ModelObjects.Stock object at 0x1043c5550>, <ModelObjects.Stock object at 0x1043c5390>, <ModelObjects.Stock object at 0x1043c55c0>, <ModelObjects.Variable object at 0x1043c5630>, <ModelObjects.Variable object at 0x1043c56a0>, <ModelObjects.Variable object at 0x1043c5710>, <ModelObjects.Variable object at 0x1043c5780>, <ModelObjects.Variable object at 0x1043c57f0>, <ModelObjects.Variable object at 0x1043c5860>, <ModelObjects.Variable object at 0x1043c5e80>, <ModelObjects.Variable object at 0x104399da0>, <ModelObjects.Variable object at 0x104399eb8>, <ModelObjects.Variable object at 0x104399f28>, <ModelObjects.Variable object at 0x104399f98>, <ModelObjects.Flow object at 0x1043c5358>, <ModelObjects.Flow object at 0x1043c5278>, <ModelObjects.Flow object at 0x1043ba0b8>]
